# Quantum Computation with Rotations!

"Stellar Representation of Grassmannians" arXiv:1909.02592v1

"Toponomic Quantum Computation" arXiv:2202.01973v1

In [1]:
import vpython as vp
import numpy as np
import scipy as sc
from scipy.special import binom
from scipy.stats import unitary_group
from itertools import combinations, permutations
from functools import reduce 
from qutip.utilities import clebsch

np.set_printoptions(precision=4, suppress=True)

## Spin Operators

In [2]:
def basis(d, i):
    v = np.zeros(d)
    v[i] = 1
    return v

def rand_ket(j):
    d = int(2*j + 1)
    v = np.random.randn(d) + 1j*np.random.randn(d) 
    return v/np.linalg.norm(v)

def spin_basis(j, m):
    v = np.zeros(int(2*j+1))
    v[int(j-m)] = 1
    return v

def sigma_plus(j):
    return np.array([[np.sqrt(j*(j+1) - m*n) \
                          if m == n+1 else 0 \
                              for n in np.arange(j, -j-1, -1)]\
                                 for m in np.arange(j, -j-1, -1)])

def sigma_minus(j):
    return sigma_plus(j).conj().T

def sigma_z(j):
    return np.diag(np.array([m for m in np.arange(j, -j-1, -1)]))

def sigma_y(j):
    return (sigma_plus(j) - sigma_minus(j))/(2j)

def sigma_x(j):
    return (sigma_plus(j) + sigma_minus(j))/2

## Majorana Polynomial

In [3]:
def majorana_polynomial(ket):
    j = (len(ket)-1)/2
    return np.polynomial.Polynomial(\
                    [(-1)**(j-m)*\
                     np.sqrt(binom(2*j, j-m))*\
                     spin_basis(j, m).conj() @ ket\
                         for m in np.arange(-j, j+1)])

def projective_roots(n, poly):
    r = poly.roots()
    return np.concatenate([r, np.repeat(np.inf, n-len(r))])

def majorana_roots(ket):
    return projective_roots(len(ket)-1, majorana_polynomial(ket))

def plane_to_sphere(z):
    if z == np.inf:
        return np.array([0,0,-1])
    else:
        return np.array([2*z.real, 2*z.imag, 1-z.real**2 - z.imag**2])/\
                    (1+z.real**2+z.imag**2)
    
def projective_stars(n, poly):
    return np.array([plane_to_sphere(r) for r in projective_roots(n, poly)])

def majorana_stars(ket):
    return projective_stars(len(ket)-1, majorana_polynomial(ket))

In [4]:
def viz_projective_stars(n, poly):
    scene = vp.canvas(background=vp.color.white)
    vp.sphere(color=vp.color.blue, opacity=0.2)
    [vp.sphere(radius=0.1, pos=vp.vector(*star)) for star in projective_stars(n, poly)]
    
def viz_majorana_stars(ket):
    viz_projective_stars(len(ket)-1, majorana_polynomial(ket))

### Example 1. A spin-2 constellation

In [5]:
psi_tetra = np.array([1,0,0,np.sqrt(2),0])/np.sqrt(3); psi_tetra

array([0.5774, 0.    , 0.    , 0.8165, 0.    ])

In [6]:
majorana_polynomial(psi_tetra)

Polynomial([ 0.    , -1.633 ,  0.    ,  0.    ,  0.5774], domain=[-1,  1], window=[-1,  1])

In [7]:
majorana_roots(psi_tetra)

array([-0.7071-1.2247j, -0.7071+1.2247j,  0.    +0.j    ,  1.4142+0.j    ])

In [8]:
majorana_stars(psi_tetra)

array([[-0.4714, -0.8165, -0.3333],
       [-0.4714,  0.8165, -0.3333],
       [ 0.    ,  0.    ,  1.    ],
       [ 0.9428,  0.    , -0.3333]])

In [50]:
viz_majorana_stars(psi_tetra)

<IPython.core.display.Javascript object>

## Some tools for Grassmannians

In [10]:
def random_grassmannian(k, n):
    return unitary_group.rvs(n)[:k]

def standard_grassmannian_form(G):
    return np.linalg.inv(G[:,:2]) @ G

def plucker_coordinate(I, G):
    return np.linalg.det(G[:, I])

def plucker_indices(k, n):
    return list(combinations(list(range(n)), k))

def plucker_coordinates(G):
    return np.array([plucker_coordinate(i, G) for i in plucker_indices(*G.shape)])

In [11]:
def __antisymmetrize__(a, b):
    return np.kron(a, b) - np.kron(b, a)

def antisymmetrize(*V):
    return reduce(__antisymmetrize__, V)

In [12]:
def plucker_basis(k, n):
    return np.array([antisymmetrize(*[basis(n, i) for i in I]) for I in plucker_indices(k, n)])

def plucker_inner_product(v, w):
    return np.linalg.det(v.conj() @ w.T)

def kplane_inner_product(v, w):
    return abs(plucker_inner_product(v,w))/\
                (np.sqrt(plucker_inner_product(v,v))*\
                 np.sqrt(plucker_inner_product(w,w)))

In [13]:
G = random_grassmannian(2, 4)
print(G)

[[-0.2713-0.0881j  0.0947-0.2594j  0.7701+0.4688j  0.1638-0.0511j]
 [-0.3165+0.4036j -0.2218+0.5055j -0.0917+0.2475j  0.1635-0.5795j]]


In [14]:
print(G @ G.conj().T)

[[ 1.+0.j -0.-0.j]
 [-0.+0.j  1.+0.j]]


In [15]:
print(standard_grassmannian_form(G))

[[ 1.    +0.j     -0.    +0.j     -1.2268-1.7925j -0.7272+0.6143j]
 [ 0.    -0.j      1.    +0.j      1.1668+1.8688j -0.284 -0.2341j]]


In [16]:
c = plucker_coordinates(G);
print(c)

[ 0.03  -0.2379j  0.4796-0.2215j -0.0642+0.0605j  0.4633-0.2381j
 -0.1243-0.1915j  0.4   -0.4149j]


In [17]:
b = plucker_basis(2,4)
print(b)

[[ 0.  1.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0. -1.  0.]]


In [18]:
print(sum([c[i]*b[i] for i in range(len(c))]))

[ 0.    +0.j      0.03  -0.2379j  0.4796-0.2215j -0.0642+0.0605j
 -0.03  +0.2379j  0.    +0.j      0.4633-0.2381j -0.1243-0.1915j
 -0.4796+0.2215j -0.4633+0.2381j  0.    +0.j      0.4   -0.4149j
  0.0642-0.0605j  0.1243+0.1915j -0.4   +0.4149j  0.    +0.j    ]


In [19]:
print(antisymmetrize(*G))

[ 0.    +0.j      0.03  -0.2379j  0.4796-0.2215j -0.0642+0.0605j
 -0.03  +0.2379j  0.    +0.j      0.4633-0.2381j -0.1243-0.1915j
 -0.4796+0.2215j -0.4633+0.2381j  0.    +0.j      0.4   -0.4149j
  0.0642-0.0605j  0.1243+0.1915j -0.4   +0.4149j  0.    +0.j    ]


## Multiconstellations for (j, k)-planes

In [20]:
def spin_operator_to_plucker(O, k):
    n = len(O)
    P = plucker_basis(k, n)
    return P @ sum([reduce(np.kron, \
                               [O if i == j else np.eye(n) \
                                   for j in range(k)])\
                                       for i in range(k)]) @ P.T/k

def lower_until_zero(j, k, highest=None):
    N = int(binom(int(2*j+1), k))
    highest = highest if type(highest) == np.ndarray else basis(N, 0)
    rungs = [highest]
    L = spin_operator_to_plucker(sigma_minus(j), k)
    while not np.allclose(rungs[-1], np.zeros(N)):
        rungs[-1] = rungs[-1]/np.linalg.norm(rungs[-1])
        rungs.append(L @ rungs[-1])
    return np.array(rungs[:-1])

### Example 4. Irreducible components for (1, 2)-planes

In [21]:
print(plucker_basis(2,3))

[[ 0.  1.  0. -1.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0. -1.  0.]]


In [22]:
lower_until_zero(1, 2)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [23]:
W = np.array([[1,0,1j], [0, 1, 1-1j]])

In [24]:
majorana_stars(W[0])

array([[-0.7071,  0.7071,  0.    ],
       [ 0.7071, -0.7071,  0.    ]])

In [25]:
plucker_coordinates(W)

array([1.+0.j, 1.-1.j, 0.-1.j])

In [26]:
majorana_stars(plucker_coordinates(W))

array([[ 0.7071, -0.7071,  0.    ],
       [ 0.7071, -0.7071, -0.    ]])

### Example 5. Irreducible components for (3/2, 2)-planes

In [27]:
print(plucker_basis(2,4))

[[ 0.  1.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0. -1.  0.]]


In [28]:
U1 = lower_until_zero(3/2, 2)
print(U1)

[[1.     0.     0.     0.     0.     0.    ]
 [0.     1.     0.     0.     0.     0.    ]
 [0.     0.     0.7071 0.7071 0.     0.    ]
 [0.     0.     0.     0.     1.     0.    ]
 [0.     0.     0.     0.     0.     1.    ]]


In [29]:
U2 = lower_until_zero(3/2, 2, np.array([0,0,1,-1,0,0])/np.sqrt(2))
print(U2)

[[ 0.      0.      0.7071 -0.7071  0.      0.    ]]


In [30]:
U_32_2 = np.concatenate([U1, U2])
print(U_32_2)

[[ 1.      0.      0.      0.      0.      0.    ]
 [ 0.      1.      0.      0.      0.      0.    ]
 [ 0.      0.      0.7071  0.7071  0.      0.    ]
 [ 0.      0.      0.      0.      1.      0.    ]
 [ 0.      0.      0.      0.      0.      1.    ]
 [ 0.      0.      0.7071 -0.7071  0.      0.    ]]


In [31]:
W = np.array([[1,0,1j,0],[0,1,0,1j]])/np.sqrt(2)

In [32]:
print(plucker_coordinates(W))

[ 0.5+0.j   0. +0.j   0. +0.5j  0. -0.5j  0. +0.j  -0.5+0.j ]


In [33]:
print(U_32_2 @ plucker_coordinates(W))

[ 0.5+0.j      0. +0.j      0. +0.j      0. +0.j     -0.5+0.j
  0. +0.7071j]


In [34]:
viz_majorana_stars((U_32_2 @ plucker_coordinates(W))[:5])

<IPython.core.display.Javascript object>

In [35]:
W = np.array([[1,0,-1j,0], [0,1,0,-1j]])/np.sqrt(2)

In [36]:
print(plucker_coordinates(W))

[ 0.5+0.j   0. +0.j   0. -0.5j  0. +0.5j  0. +0.j  -0.5+0.j ]


In [37]:
print(U_32_2 @ plucker_coordinates(W))

[ 0.5+0.j      0. +0.j      0. +0.j      0. +0.j     -0.5+0.j
  0. -0.7071j]


In [38]:
viz_majorana_stars((U_32_2 @ plucker_coordinates(W))[:5])

<IPython.core.display.Javascript object>

### Example 6. Multiconstellation for a (2, 2)-plane

In [39]:
U1 = lower_until_zero(2,2)
U2 = lower_until_zero(2,2,np.array([0,0,np.sqrt(2/5), 0, -np.sqrt(3/5),0,0,0,0,0]))
U_2_2 = np.concatenate([U1, U2])
print(U_2_2)

[[ 1.      0.      0.      0.      0.      0.      0.      0.      0.
   0.    ]
 [ 0.      1.      0.      0.      0.      0.      0.      0.      0.
   0.    ]
 [ 0.      0.      0.7746  0.      0.6325  0.      0.      0.      0.
   0.    ]
 [ 0.      0.      0.      0.4472  0.      0.8944  0.      0.      0.
   0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.7746  0.6325  0.
   0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      1.
   0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      0.
   1.    ]
 [ 0.      0.      0.6325  0.     -0.7746  0.      0.      0.      0.
   0.    ]
 [ 0.      0.      0.      0.8944  0.     -0.4472  0.      0.      0.
   0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.6325 -0.7746  0.
   0.    ]]


In [40]:
W = np.array([[1,0,1,0,0],[0,1,0,0,1]])/np.sqrt(2)

In [41]:
print(plucker_coordinates(W))

[ 0.5  0.   0.   0.5 -0.5  0.   0.   0.   0.5  0. ]


In [42]:
BD = U_2_2 @ plucker_coordinates(W); print(BD)

[ 0.5     0.     -0.3162  0.2236  0.      0.5     0.      0.3873  0.4472
  0.    ]


In [43]:
spin3 = BD[:7]; spin1 = BD[7:]

### Finding the spectator constellation

In [44]:
def spherical_tensor(j, l, m):
    return np.array([[(-1.)**(j-m1-m)*clebsch(j, j, l, m1, -m2, m) \
                   for m1 in np.arange(j, -j-1, -1)]\
                          for m2 in np.arange(j, -j-1, -1)])

def multipole_expansion(O):
    j = (len(O)-1)/2
    return np.array([[(spherical_tensor(j, l, m).conj().T @ O).trace() for m in np.arange(-l, l+1)] for l in np.arange(0, 2*j+1)])

In [45]:
def expect_xyz(ket):
    j = (len(ket)-1)/2
    return np.array([ket.conj() @ sigma_x(j) @ ket,\
                     ket.conj() @ sigma_y(j) @ ket,\
                     ket.conj() @ sigma_z(j) @ ket]).real

def rotate(j, theta, phi):
    return sc.linalg.expm((theta/2) * \
                          (np.exp(-1j*phi)*sigma_plus(j) -\
                           np.exp(1j*phi)*sigma_minus(j)))

def cartesian_to_spherical(xyz):
    x, y, z = xyz
    r = np.linalg.norm(xyz)
    return np.array([r,np.arccos(z/r),np.arctan2(y,x)])

In [46]:
def first_nonzero_in_multipole_expansion(ket):
    j = (len(ket)-1)/2
    MP = multipole_expansion(np.outer(ket, ket.conj()))
    for l in np.arange(0, 2*j+1):
        for m in np.arange(-l, l+1):
            c = MP[int(l)][int(m+l)]
            if not np.isclose(c,0) and m != 0:
                return [c, l, m]
            
def spectator_component(ket):
    if len(ket) == 1:
        return ket[0]
    j = (len(ket)-1)/2
    r, theta, phi = cartesian_to_spherical(expect_xyz(ket))
    ket_ = rotate(j, theta, phi) @ ket
    c, l, m = first_nonzero_in_multipole_expansion(ket_)
    ket__ = sc.linalg.expm(-1j*np.angle(c)*sigma_z(j)/m) @ ket_
    c2 = ket__[np.where(np.logical_not(np.isclose(ket__, 0)))[0][0]]
    return np.sqrt(ket.conj() @ ket)*np.exp(1j*np.angle(c2))

### By hand, for the spin-3 part

In [47]:
expect_xyz(spin3)

array([-0.2449,  0.    ,  0.35  ])

In [48]:
r, theta, phi = cartesian_to_spherical(expect_xyz(spin3)); theta, phi

(0.6106281135425041, 3.141592653589793)

In [51]:
spin3_ = rotate(3, theta, phi) @ spin3

In [52]:
multipole_expansion(np.outer(spin3_, spin3_.conj()))

array([list([(0.24567690745599757+0j)]),
       list([(6.938893903907228e-18+4.008597611912941e-18j), (0.08073324682469504+0j), (-6.938893903907228e-18+4.008597611912942e-18j)]),
       list([(0.011349449999020763+3.545966333935906e-18j), (0.043344649430436824+6.402920632522105e-18j), (0.05403134747966522+0j), (-0.043344649430436824+6.402920632522105e-18j), (0.011349449999020763-3.545966333935906e-18j)]),
       list([(-0.08993661123520022-2.2372105749071587e-18j), (0.043417288866579175+9.84602891805298e-18j), (0.14103797950199523-4.383216340710986e-18j), (-0.1013373901874251+0j), (-0.14103797950199523-4.383216340710986e-18j), (0.043417288866579175-9.84602891805298e-18j), (0.08993661123520022-2.2372105749071587e-18j)]),
       list([(0.033428586304941776-1.3193812744834549e-18j), (-0.04308546177938187-1.2064705686510923e-17j), (-0.09533361836754878-4.872422667621215e-18j), (0.033615351535922725-4.028725231026191e-17j), (-0.1708794296759276+0j), (-0.033615351535922725-4.028725231026192e

In [53]:
spin3_

array([ 0.2583-0.j,  0.5813+0.j, -0.1198-0.j,  0.0847+0.j, -0.2787-0.j,
        0.23  -0.j,  0.3054-0.j])

In [54]:
np.sqrt(spin3.conj() @ spin3)

0.8062257748298548

Or use:

In [55]:
spectator_component(spin3)

(0.8062257748298548+2.7535412998904975e-16j)

### By hand, for the spin-1 part

In [56]:
expect_xyz(spin1)

array([0.2449, 0.    , 0.15  ])

In [57]:
r, theta, phi = cartesian_to_spherical(expect_xyz(spin1)); theta, phi

(1.0213290820372694, 0.0)

In [58]:
spin1_ = rotate(1, theta, phi) @ spin1

In [59]:
multipole_expansion(np.outer(spin1_, spin1_.conj()))

array([list([(0.20207259421636883+0j)]),
       list([(-3.0404709722440573e-17+0j), (0.20310096011589887+0j), (3.0404709722440573e-17+0j)]),
       list([(-0.09999999999999992+0j), (-5.822058658345459e-17+0j), (0.14288690166235196+0j), (5.822058658345459e-17+0j), (-0.09999999999999992+0j)])],
      dtype=object)

In [60]:
spin1__ = sc.linalg.expm(1j*np.pi*sigma_z(1)/2) @ spin1_; spin1__

array([ 0.+0.5645j, -0.+0.j    ,  0.+0.1772j])

In [61]:
np.sqrt(spin1.conj() @ spin1) * np.exp(1j*np.angle(spin1__[0]))

(3.6225540849366556e-17+0.5916079783099615j)

Or use:

In [62]:
spectator_component(spin1)

(3.6225540849366556e-17+0.5916079783099615j)

### Putting it together:

In [63]:
viz_majorana_stars(spin3)

<IPython.core.display.Javascript object>

In [64]:
viz_majorana_stars(spin1)

<IPython.core.display.Javascript object>

In [65]:
spectator = np.array([spectator_component(spin3), spectator_component(spin1)]); spectator

array([0.8062+0.j    , 0.    +0.5916j])

In [66]:
viz_majorana_stars(spectator)

<IPython.core.display.Javascript object>

## Holonomies

In [67]:
def check_1anticoherence(G):
    k, n = G.shape
    j = (n-1)/2
    return np.allclose(np.array([G @ sigma_x(j) @ G.conj().T,\
                                 G @ sigma_y(j) @ G.conj().T,\
                                 G @ sigma_z(j) @ G.conj().T]),0)

In [68]:
def qmatrix(f, t):
    k = len(f(0))
    return np.array([[f(0)[i].conj() @ f(t)[j] for j in range(k)] for i in range(k)])

def holonomy(f, t):
    U, D, V = np.linalg.svd(qmatrix(f, t))
    return U @ V

def rot_holonomy(G, R):
    k = G.shape[0]
    G_ = G @ R
    Q = np.array([[G[i].conj() @ G_[j] for j in range(k)] for i in range(k)])
    U, D, V = np.linalg.svd(Q)
    return U @ V

### Example 1: a spin-2 2 plane

In [69]:
psi1 = np.array([1,0,0,np.sqrt(2),0])/np.sqrt(3)
viz_majorana_stars(psi1)

<IPython.core.display.Javascript object>

In [70]:
psi2 = np.array([0,-np.sqrt(2),0,0,1])/np.sqrt(3)
viz_majorana_stars(psi2)

<IPython.core.display.Javascript object>

In [71]:
pi_not = np.array([psi1, psi2])

In [72]:
check_1anticoherence(pi_not)

True

In [73]:
pi_not_evolution = lambda t: pi_not @ sc.linalg.expm(-1j*sigma_y(2)*np.pi*t)
pi_not_holonomy = holonomy(pi_not_evolution, 1); pi_not_holonomy

array([[ 0.+0.j,  1.+0.j],
       [ 1.+0.j, -0.+0.j]])

In [74]:
rot_holonomy(pi_not, sc.linalg.expm(-1j*sigma_y(2)*np.pi))

array([[ 0.+0.j,  1.+0.j],
       [ 1.+0.j, -0.+0.j]])

In [75]:
pi_not_holonomy @ pi_not_evolution(0)

array([[ 0.    +0.j, -0.8165+0.j,  0.    +0.j,  0.    +0.j,  0.5774+0.j],
       [ 0.5774+0.j,  0.    +0.j,  0.    +0.j,  0.8165+0.j, -0.    +0.j]])

In [76]:
pi_not @ sc.linalg.expm(-1j*sigma_y(2)*np.pi)

array([[ 0.    +0.j, -0.8165+0.j, -0.    +0.j,  0.    +0.j,  0.5774+0.j],
       [ 0.5774+0.j,  0.    +0.j, -0.    +0.j,  0.8165+0.j,  0.    +0.j]])

In [77]:
plucker_coordinates(pi_not)

array([-0.4714,  0.    ,  0.    ,  0.3333,  0.    ,  0.6667,  0.    ,
        0.    ,  0.    ,  0.4714])

In [78]:
bd_pi_not = U_2_2 @ plucker_coordinates(pi_not); bd_pi_not

array([-0.4714,  0.    ,  0.    ,  0.7454,  0.    ,  0.    ,  0.4714,
        0.    ,  0.    ,  0.    ])

In [79]:
pi_not_spin3 = bd_pi_not[0:7]
viz_majorana_stars(pi_not_spin3)

<IPython.core.display.Javascript object>

#### So:

In [80]:
test = np.sqrt(1/3)*psi1 + np.sqrt(2/3)*psi2; test

array([ 0.3333, -0.6667,  0.    ,  0.4714,  0.4714])

In [81]:
sc.linalg.expm(-1j*sigma_y(2)*np.pi) @ test

array([ 0.4714+0.j, -0.4714+0.j,  0.    +0.j,  0.6667+0.j,  0.3333+0.j])

In [82]:
np.sqrt(2/3)*psi1 + np.sqrt(1/3)*psi2

array([ 0.4714, -0.4714,  0.    ,  0.6667,  0.3333])

### Example 2: a spin-5 4 plane

In [83]:
pi_cnot = np.array([(spin_basis(5,5) + 1j*np.sqrt(2)*spin_basis(5,0) + spin_basis(5,-5))/2,\
                    (spin_basis(5,5) - 1j*np.sqrt(2)*spin_basis(5,0) + spin_basis(5,-5))/2,\
                    (np.sqrt(2)*spin_basis(5,3) + 1j*np.sqrt(3)*spin_basis(5,-2) )/np.sqrt(5),\
                    (1j*np.sqrt(3)*spin_basis(5,2) + np.sqrt(2)*spin_basis(5,-3))/np.sqrt(5)])

In [84]:
check_1anticoherence(pi_cnot)

True

In [85]:
pi_cnot_holonomy1 = rot_holonomy(pi_cnot, sc.linalg.expm(-1j*sigma_x(5)*np.pi)); pi_cnot_holonomy1

array([[-1.+0.j, -0.+0.j, -0.+0.j,  0.+0.j],
       [ 0.+0.j, -1.+0.j, -0.+0.j, -0.+0.j],
       [-0.+0.j,  0.+0.j,  0.+0.j, -1.+0.j],
       [ 0.+0.j,  0.+0.j, -1.+0.j, -0.+0.j]])

In [86]:
np.allclose(pi_cnot_holonomy1 @ pi_cnot, pi_cnot @  sc.linalg.expm(-1j*sigma_x(5)*np.pi))

True

In [87]:
pi_cnot_holonomy2 = rot_holonomy(pi_cnot, sc.linalg.expm(-1j*sigma_z(5)*(2*np.pi/5))); pi_cnot_holonomy2

array([[ 1.   +0.j    ,  0.   +0.j    ,  0.   +0.j    ,  0.   +0.j    ],
       [-0.   +0.j    ,  1.   +0.j    ,  0.   +0.j    ,  0.   +0.j    ],
       [ 0.   +0.j    ,  0.   +0.j    , -0.809+0.5878j,  0.   +0.j    ],
       [ 0.   +0.j    ,  0.   +0.j    ,  0.   +0.j    , -0.809-0.5878j]])

In [88]:
np.allclose(pi_cnot_holonomy2 @ pi_cnot, pi_cnot @  sc.linalg.expm(-1j*sigma_z(5)*(2*np.pi/5)))

True

## The Principle Constellation of a (j, k) plane

In [89]:
def sign(lst):
    parity = 1
    for i in range(0,len(lst)-1):
        if lst[i] != i:
            parity *= -1
            mn = min(range(i,len(lst)), key=lst.__getitem__)
            lst[i],lst[mn] = lst[mn],lst[i]
    return parity    

def principle_constellation_polynomial(G):
    k, n = G.shape
    P = [majorana_polynomial(g) for g in G]
    W = np.array([[P[i].deriv(m=j) for j in range(k)] for i in range(k)])
    return sum([sign(list(perm))*np.prod([W[i,p] for i, p in enumerate(perm)]) for perm in permutations(list(range(k)))]) 

def number_of_principal_stars(j, k):
    return 2*np.sum(np.arange(j-k+1, j+1))

In [90]:
principle_constellation_polynomial(pi_cnot)

Polynomial([     0.          +0.j    ,      0.          +0.j    ,
            0.          +0.j    ,      0.          +0.j    ,
       -72737.8196      +0.j    ,      0.          +0.j    ,
            0.          +0.j    ,      0.          +0.j    ,
            0.          +0.j    ,      0.    +2230626.4677j,
            0.          +0.j    ,      0.          +0.j    ,
            0.          +0.j    ,      0.          +0.j    ,
            0.          +0.j    ,      0.          +0.j    ,
            0.          +0.j    ,      0.          +0.j    ,
            0.          +0.j    ,      0.    -2230626.4677j,
            0.          +0.j    ,      0.          +0.j    ,
            0.          +0.j    ,      0.          +0.j    ,
        72737.8196      +0.j    ], domain=[-1.,  1.], window=[-1.,  1.])

In [91]:
viz_projective_stars(number_of_principal_stars(5,4),\
                     principle_constellation_polynomial(pi_cnot))

<IPython.core.display.Javascript object>

## Coherent States and Coherent (j,k)-planes

In [92]:
def spin_coherent_state(j, z):
    if z == np.inf:
        return spin_coherent_state(j, 0)[::-1]
    return (1+z*z.conjugate())**(-j) * \
            sum([np.sqrt(binom(2*j, j-m))*z**(j-m)*spin_basis(j,m)\
                     for m in np.arange(-j, j+1)])

In [93]:
z = 1.1
spin_coherent_state(3/2, z)

array([0.3044, 0.5799, 0.6379, 0.4051])

In [94]:
cs = sc.linalg.expm(z*sigma_minus(3/2)) @ spin_basis(3/2,3/2)
cs = cs/np.linalg.norm(cs); cs

array([0.3044, 0.5799, 0.6379, 0.4051])

In [95]:
def integer(x):
    return np.equal(np.mod(x, 1), 0)

def spin_coherent_representation(ket):
    j = (len(ket)-1)/2
    m = majorana_polynomial(ket)
    def scs_rep(z):
        if z == np.inf:
            return ket[0]
        elif np.isclose(z, 0):
            return ket[-1]
        return ((z.conjugate()/z)/(1+z*z.conjugate()))**j * m(z)
    return scs_rep

In [96]:
j = 1/2
ket = rand_ket(j)
scs_rep1 = spin_coherent_representation(ket)
scs_rep2 = lambda z: spin_coherent_state(j, np.inf if np.isclose(z, 0) else -1/z.conjugate()).conj() @ ket

In [97]:
scs_rep1(1), scs_rep2(1)

((-0.39475393139254256-0.6390879433575467j),
 (-0.39475393139254256-0.6390879433575467j))

In [98]:
scs_rep1(0), scs_rep2(0)

((0.19196489707626369+0.9104551138559811j),
 (0.19196489707626369+0.9104551138559811j))

In [99]:
scs_rep1(np.inf), scs_rep2(np.inf)

((-0.3663014664991682+0.0066482768106103166j),
 (-0.3663014664991682+0.0066482768106103166j))

## Spin Coherent planes <=> Principal Constellations

In [100]:
def coherent_plane(j, k):
    d = int(2*j+1)
    return lambda z: np.eye(d)[:k,:]@sc.linalg.expm(-z*sigma_plus(j)) 

In [101]:
coherent_plane(3/2, 2)(1)

array([[ 1.    , -1.7321,  1.7321, -1.    ],
       [ 0.    ,  1.    , -2.    ,  1.7321]])

In [102]:
w = np.array([[1,0,0,np.sqrt(2)], [0,1,0,0]])

In [103]:
# Note that we use 1/z.conjugate() instead of -1/z.conjugate()... why?
pcp = lambda z: z**number_of_principal_stars(3/2,2)*\
        plucker_inner_product(coherent_plane(3/2, 2)(1/z.conjugate()), w)

In [104]:
pcp(1)

3.8284271247461885

In [105]:
# Why the sqrt(3)?
principle_constellation_polynomial(w)(1)/np.sqrt(3)

3.8284271247461907

## Finding 1-Anticoherent Subspaces

In [106]:
import jax
import jax.numpy as jp
from jax.config import config
config.update("jax_enable_x64", True)

In [107]:
def find_1anticoherent_subspace(j, k, max_iter=1000):
    d = int(2*j + 1)
    X, Y, Z = sigma_x(j), sigma_y(j), sigma_z(j)

    @jax.jit
    def one_anticoherence(V):
        R = (V[0:d*k] + 1j*V[d*k:]).reshape(k, d)
        return (jp.linalg.norm(R @ X @ R.conj().T) + \
                jp.linalg.norm(R @ Y @ R.conj().T) + \
                jp.linalg.norm(R @ Z @ R.conj().T)).real

    @jax.jit
    def orthogonality(V):
        R = (V[0:d*k] + 1j*V[d*k:]).reshape(k, d)
        return jp.linalg.norm((R @ R.conj().T) - jp.eye(k)).real
    
    for t in range(max_iter):
        try:
            V = np.random.randn(2*d*k)
            result = sc.optimize.minimize(one_anticoherence, V,\
                                          jac=jax.jit(jax.jacrev(one_anticoherence)),\
                                          tol=1e-23,\
                                          constraints=[{"type": "eq",\
                                                        "fun": orthogonality,\
                                                        "jac": jax.jit(jax.jacrev(orthogonality))}],\
                                          options={"disp": True,\
                                                   "maxiter": 5000},
                                          method="trust-constr")
            R = (result.x[0:d*k] + 1j*result.x[d*k:]).reshape(k, d)
            return R
        except:
            continue

In [108]:
j, k = 2, 2
R = find_1anticoherent_subspace(j, k)

`xtol` termination condition is satisfied.
Number of iterations: 170, function evaluations: 237, CG iterations: 236, optimality: 2.44e+00, constraint violation: 1.39e-17, execution time: 0.18 s.


/Users/heyredhat/opt/anaconda3/envs/spheres/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


In [109]:
R @ R.conj().T

array([[1.+0.j, 0.-0.j],
       [0.+0.j, 1.+0.j]])

In [110]:
R @ sigma_x(j) @ R.conj().T

array([[ 0.+0.j, -0.+0.j],
       [-0.+0.j,  0.+0.j]])

In [111]:
R @ sigma_y(j) @ R.conj().T

array([[-0.-0.j,  0.-0.j],
       [ 0.+0.j,  0.+0.j]])

In [112]:
R @ sigma_z(j) @ R.conj().T

array([[ 0.+0.j,  0.+0.j],
       [ 0.+0.j, -0.+0.j]])

In [113]:
bd = U_2_2 @ plucker_coordinates(R)
j3 = bd[:7]
j1 = bd[7:]
(j3, j1)

(array([ 0.2349+0.316j ,  0.1549-0.2546j,  0.4767+0.1486j,  0.1158+0.0132j,
        -0.4979+0.0374j,  0.0936+0.283j , -0.3001+0.2549j]),
 array([-0.-0.j,  0.+0.j, -0.-0.j]))

In [114]:
viz_majorana_stars(j3)

<IPython.core.display.Javascript object>

## Identifying rotational symmetries

To be continued!